In [19]:
from pathlib import Path

In [20]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
if RunningInCOLAB:
  my_file = Path("trainm")
  if my_file.exists()==False:
    !unzip drive/"My Drive"/trainm.zip

In [36]:
if RunningInCOLAB:
  kaggle_folder = Path(".kaggle")
  if Path("rvdata").exists()==False:
    !pip install -q kaggle
    !mkdir ~/.kaggle
    !cp drive/"My Drive"/kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset
    !unzip real-life-violence-situations-dataset.zip
    !rm real-life-violence-situations-dataset.zip
    !rm -R "real life violence situations"
    !mv -T "Real Life Violence Dataset" "rvdata"
    !kaggle datasets download -d vulamnguyen/rwf2000
    !unzip rwf2000.zip
    !rm rwf2000.zip
    !mv RWF-2000/train/* RWF-2000/
    !mv RWF-2000/val/Fight/* RWF-2000/Fight
    !mv RWF-2000/val/NonFight/* RWF-2000/NonFight
    !rm -R RWF-2000/train/
    !rm -R RWF-2000/val/
    #!git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos.git
    #!mv -T A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset vdd
    #!rm -R A-Dataset-for-Automatic-Violence-Detection-in-Videos
    !kaggle datasets download -d toluwaniaremu/smartcity-cctv-violence-detection-dataset-scvd
    !unzip smartcity-cctv-violence-detection-dataset-scvd.zip
    !rm smartcity-cctv-violence-detection-dataset-scvd.zip
    !rm -R SCVD/SCVD_converted_sec_split
    !mkdir SCVD/NonViolence
    !mkdir SCVD/Violence
    !mv SCVD/SCVD_converted/Train/Normal/* SCVD/NonViolence
    !mv SCVD/SCVD_converted/Train/Violence/* SCVD/Violence
    !mv SCVD/SCVD_converted/Train/Weaponized/* SCVD/Violence
    !mv SCVD/SCVD_converted/Test/Normal/* SCVD/NonViolence
    !mv SCVD/SCVD_converted/Test/Violence/* SCVD/Violence
    !mv SCVD/SCVD_converted/Test/Weaponized/* SCVD/Violence
    !rm -R SCVD/SCVD_converted

Streaming output truncated to the last 5000 lines.
  inflating: RWF-2000/train/Fight/rfthfd_536.avi  
  inflating: RWF-2000/train/Fight/rfthfd_537.avi  
  inflating: RWF-2000/train/Fight/rfthfd_538.avi  
  inflating: RWF-2000/train/Fight/rn7Qjaj9_0.avi  
  inflating: RWF-2000/train/Fight/rn7Qjaj9_1.avi  
  inflating: RWF-2000/train/Fight/s-jW4cyavW0_0.avi  
  inflating: RWF-2000/train/Fight/s-jW4cyavW0_1.avi  
  inflating: RWF-2000/train/Fight/s-jW4cyavW0_2.avi  
  inflating: RWF-2000/train/Fight/s3gaPKuW_0.avi  
  inflating: RWF-2000/train/Fight/s3gaPKuW_1.avi  
  inflating: RWF-2000/train/Fight/sadffg_950.avi  
  inflating: RWF-2000/train/Fight/sadsa_956.avi  
  inflating: RWF-2000/train/Fight/sadsa_957.avi  
  inflating: RWF-2000/train/Fight/sadsa_958.avi  
  inflating: RWF-2000/train/Fight/sadsa_959.avi  
  inflating: RWF-2000/train/Fight/sadsa_960.avi  
  inflating: RWF-2000/train/Fight/sadsa_961.avi  
  inflating: RWF-2000/train/Fight/t2z75yT2S_M_0.avi  
  inflating: RWF-2000/tra

In [23]:
#!pip install tensorflow==2.18.rc2

In [24]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from numpy import save
from numpy import load
from sklearn.model_selection import train_test_split
random = np.random
layers = tf.keras.layers
models = tf.keras.models
losses = tf.keras.losses
optimizers = tf.keras.optimizers
metrics = tf.keras.metrics
utils = tf.keras.utils
callbacks = tf.keras.callbacks
layers = tf.keras.layers
models = tf.keras.models
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
losses = tf.keras.losses
optimizers = tf.keras.optimizers
metrics = tf.keras.metrics
utils = tf.keras.utils
callbacks = tf.keras.callbacks
plot_model = tf.keras.utils.plot_model

In [25]:
mono = False
channel_first = False
frame_readed = 500
frame_pages = 20
nbr_frame = 8
img_width = 420
img_height = 300
img_size = (img_height, img_width)
input_shape = (3,) + img_size if channel_first else img_size + (3,)
full_input_shape = (nbr_frame,) + input_shape
print(full_input_shape)

(8, 300, 420, 3)


In [26]:
input_format = 'channels_first' if channel_first else 'channels_last'
tf.keras.backend.set_image_data_format(
      input_format
)

def capture(filename):
  frames = np.zeros( (frame_readed,) +input_shape, dtype=np.float32)
  i = 0
  vc = cv2.VideoCapture(filename)
  while i<frame_readed:
      if vc.isOpened():
          rval, frame = vc.read()
      else:
          print( 'Failed to read the video ' + filename)
          return frames
      if rval==False:
          break
      frm = cv2.resize(frame, (img_width, img_height))
      frm = tf.keras.applications.efficientnet.preprocess_input(frm)
      frm = np.expand_dims(frm, axis=0)
      #if np.max(frm)>1:
      #  frm = frm/255.0
      if channel_first:
        frm = frm.transpose(0,3,1,2)
      frames[i][:] = frm
      i +=1
      if i==(nbr_frame*30):
          break
  vc.release()
  step = int(i/nbr_frame)
  if (step>4):
    step = random.randint(2, step)
  start = int((i - step*nbr_frame)/2)
  start += random.randint(0, int(start/2)+1 ) - int(start/2)
  if (start<0):
    start = 0
  if (start + step*nbr_frame > i):
    start = i - step*nbr_frame
  resf = np.empty( (nbr_frame,) + input_shape )
  k = 0
  while k<nbr_frame:
      j = int(step*k)
      frm = frames[start+j]
      resf[k][:] = frm
      k = k+1
  return resf


In [27]:
#cut_save('./testm/',"test")
#plt.imshow(fights_test[19][5])
#plt.show()

In [28]:
class AccuracyHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []
        self.loss = []
        self.val_loss = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

history = AccuracyHistory()
earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=8,min_delta=1e-5, verbose=0, mode='min')
mcp_save = callbacks.ModelCheckpoint('fights.keras', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='val_loss',patience=10, verbose=2, factor=0.5,min_lr=0.0000001)


In [29]:
maxSize = 10*1024*1024
def loadData(dataset, ist, ito, fts_train, labs_train):
  print('Loading ' + dataset +' from ' + str(ist) + ' to ' + str(ito) )
  page_size = ito-ist
  i= 0
  for x in os.listdir('./' + dataset):
    lx = x.lower()
    violence = 1 if (lx.startswith('fight') or lx.startswith('viol')) else 0
    td = dataset+'/'+x+'/'
    notGood = 'fight' not in x.lower() and 'viol' not in x.lower()
    if notGood or os.path.isdir(td)==False:
      continue
    n= 0
    #print('-> folder ' +td)
    nbrFiles = 0
    for file in Path(td).rglob("*"):
      fl = os.path.join('', file)
      #print('file ' + fl)
      #print(os.path.getsize(fl))
      if file.is_dir():
        continue
      if os.path.getsize(fl)>maxSize:
        continue
      nbrFiles += 1
    #print('nbrFiles '+str(nbrFiles))
    if ito > nbrFiles:
      ito = nbrFiles
      ist = nbrFiles-page_size
      print( 'ist - ito ' + str(ist) + ' - ' + str(ito) )
    for file in Path(td).rglob("*"):
      fl = os.path.join('', file)
      if file.is_dir():
        continue
      if os.path.getsize(fl)>maxSize:
        continue
      #print('visiting ' + fl)
      if n<ist:
        n+=1
        continue
      if n>=ito:
        break
      videos = capture(fl)
      fts_train[i][:][:] = videos
      labs_train[i] = violence
      i+=1
      n+=1
    ###### this part is not usefull anymore
    print("Readed "+str(i))
    half= int(len(fts_train)/2)
    if i<half:
      while i<half:
        print("Random repeat "+str(i)+'/'+str(half))
        p = np.random.randint(0, i-1)
        fts_train[i][:][:] = fts_train[p]
        labs_train[i] = labs_train[p]
        i+=1
    else:
      if i>half:
        while i<len(fts_train):
          print("Random repeat "+str(i)+'/'+str(half))
          p = np.random.randint(half+1, len(fts_train)-1)
          fts_train[i][:][:] = fts_train[p]
          labs_train[i] = labs_train[p]
          i+=1

In [30]:
np.random.seed(1234)
num_classes = 2
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape = input_shape,
    pooling=None,
    classes = num_classes
)
base_model.trainable = False
for layer in base_model.layers:
    layer.trainable = False
#base_model.summary()
cnn = models.Sequential()
cnn.add(base_model)
cnn.add(layers.GlobalAveragePooling2D())
def func(x):
  return tf.cast(tf.math.logical_and(x < 0.1, x > -0.1), dtype=tf.float32);
    #abs = tf.keras.backend.abs(abs, x) #will return boolean values
    #greater = tf.keras.backend.greater(abs, 0.009)
    #return tf.keras.backend.cast(greater, dtype=tf.keras.backend.floatx()) #will return boolean values
thresholdLayer = tf.keras.layers.Lambda(func)
def filters(x, i):
  #zero = tf.keras.backend.constant(np.full((1, 340, 460, 3), 0 ), 'float32')
 # x = layers.subtract(inputs=[input, avg])
 # x = thresholdLayer(x)
 # x = layers.multiply(inputs=[x, input])
  x = base_model(x)
 # k = tf.keras.backend.constant(np.full((1, 960), 1+(0.05*i) ), 'float32')
  x = layers.GlobalAveragePooling2D()(x)
  #x = layers.multiply(inputs=[x, k])
  #print(x.shape)
  return x
if mono:
   # inputLayer = layers.Input(shape=full_input_shape)
    layer = models.Sequential()
    layer.add(layers.TimeDistributed(cnn, input_shape=full_input_shape))
    layer.add(layers.GRU(nbr_frame, return_sequences=True))
    layer.add(layers.TimeDistributed(layers.Dense(nbr_frame, activation='relu')))
    layer.add(layers.Flatten())
  #  layer.add(layers.Dense(460, activation='sigmoid'))
  #  layer.add(layers.Dropout(0.1))
    layer.add(layers.Dense(420, activation='relu'))
    layer.add(layers.Dropout(0.1))
    layer.add(layers.Dense(120, activation='sigmoid'))
    layer.add(layers.Dropout(0.1))
    layer.add(layers.Dense(num_classes, activation="sigmoid"))
else:
    inputs = []
    fstLayers = []
    for i in range(nbr_frame):
      input = layers.Input(shape = input_shape, name = 'inp'+str(i+1))
      inputs.append(input)
      fstLayers.append(filters(input, 1))
    layer = layers.concatenate(fstLayers)
    layer = layers.Flatten()(layer)

    layer = layers.Dense(120, activation='sigmoid')(layer)
    layer = layers.Dropout(0.1)(layer)
    layer = layers.Dense(64, activation='relu')(layer)
    layer = layers.Dropout(0.1)(layer)
    layer = layers.Dense(20, activation='sigmoid')(layer)
    layer = layers.Dropout(0.1)(layer)
    layer = layers.Dense(num_classes, activation="sigmoid")(layer)
    print(layer.shape)

(None, 2)


In [31]:
if False:
  v=filters(input2, 1)
  m=models.Model([input1, input2, input3, input4,
                          input5, input6,
                          ], v)
  m.compile(
      loss=loss,
      optimizer= adam,
      metrics=metrics
  )
  #m.summary()
  f = m.predict([_fights_train[0], _fights_train[1], _fights_train[2],
            _fights_train[3], _fights_train[4], _fights_train[5]])
  def fix(img):
    return (img+1)*128
  print(_fights_train[4].shape)
  cv2_imshow(fix(f[4]))
  cv2_imshow(fix(_fights_train[0][4]))
  cv2_imshow(fix(_fights_train[1][4]))
  cv2_imshow(fix(_fights_train[2][4]))
  cv2_imshow(fix(_fights_train[3][4]))
  cv2_imshow(fix(_fights_train[4][4]))
  cv2_imshow(fix(_fights_train[5][4]))

In [32]:
adam = optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=1e-08)
rms = optimizers.RMSprop()
metrics = [tf.keras.metrics.CategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.CategoricalCrossentropy()

if mono:
  model = layer
else:
  model = models.Model(inputs, layer)
#model = models.Model([input1], layer)
model.compile(
    loss=loss,
    optimizer= adam,
    metrics=metrics
)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ inp1 (InputLayer)         │ (None, 300, 420, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inp2 (InputLayer)         │ (None, 300, 420, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inp3 (InputLayer)         │ (None, 300, 420, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inp4 (InputLayer)         │ (None, 300, 420, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inp5 (InputLayer)         │ (None, 300, 420, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inp6 (InputLayer)         │ (None, 300, 420, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inp7 (InputLayer)         │ (None, 300, 420, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inp8 (InputLayer)         │ (None, 300, 420, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ efficientnetb0            │ (None, 10, 14, 1280)   │      4,049,571 │ inp1[0][0],            │
│ (Functional)              │                        │                │ inp2[0][0],            │
│                           │                        │                │ inp3[0][0],            │
│                           │                        │                │ inp4[0][0],            │
│                           │                        │                │ inp5[0][0],            │
│                           │                        │                │ inp6[0][0],            │
│                           │                        │                │ inp7[0][0], inp8[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1280)           │              0 │ efficientnetb0[1][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1280)           │              0 │ efficientnetb0[2][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1280)           │              0 │ efficientnetb0[3][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1280)           │              0 │ efficientnetb0[4][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1280)           │              0 │ efficientnetb0[5][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├──────────────────────

 Total params: 5,287,577 (20.17 MB)

 Trainable params: 1,238,006 (4.72 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [33]:
!pip install git+https://github.com/onnx/tensorflow-onnx

  Cloning https://github.com/onnx/tensorflow-onnx to /tmp/pip-req-build-0vm0kwib
  Running command git clone --filter=blob:none --quiet https://github.com/onnx/tensorflow-onnx /tmp/pip-req-build-0vm0kwib
  Resolved https://github.com/onnx/tensorflow-onnx to commit 79cedf926380cdf494dbfc6dd01eaa66122616d2
  Preparing metadata (setup.py) ... done


In [34]:
import tf2onnx
import onnx

In [38]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Started at =", current_time)
datasets = [ 'trainm', 'SCVD', 'RWF-2000',  'rvdata']

dpages = [ 150, 100, 200,  200]
dsizes = [ 150, 100, 800, 1000]
nbr_it = 3
batch_size = 20
epochs = 8
for it in range(nbr_it):
  print('#### IT '+str(it))
  dt_index = 0
  for dataset in datasets:
    page_size = dpages[dt_index]
    size = dsizes[dt_index]
    pages = int(size/page_size)
    dt_index+=1
    for k in range(pages):
      fts_train = np.zeros((2*page_size,) + full_input_shape, dtype=np.float32)
      labs_train = np.empty(2*page_size, dtype=np.int_)
      loadData( dataset, k*page_size, (k+1)*page_size, fts_train, labs_train)
      random_state = np.random.randint(1, 100)
      X_train, fights_test, y_train, labels_test = train_test_split(fts_train,
                          labs_train, test_size=0.2, random_state=random_state)
      #print(labs_train)
      y_train = utils.to_categorical(labs_train, num_classes=num_classes)
      y_test = utils.to_categorical(labels_test, num_classes=num_classes)
      from datetime import datetime
      import time
      millis = int(round(time.time() * 1000))
      if mono:
        model.fit(fts_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
              #    verbose=1,
                  validation_data=(fights_test, y_test),
                  callbacks=[earlyStopping, reduce_lr_loss,history])
      else:
        _fights_train = fts_train.transpose(1,0,2,3,4)
        _fights_test = fights_test.transpose(1,0,2,3,4)
        print(_fights_train.shape)
        if True:
            fls = _fights_train
            fts = _fights_test
            print(_fights_train[0].shape)
            model.fit([fls[0], fls[1], fls[2], fls[3], fls[4], fls[5], fls[6], fls[7]], y_train,
                  batch_size=batch_size,
                  epochs=epochs,
              #    verbose=1,
                  validation_data=([fts[0], fts[1], fts[2], fts[3],
                                    fts[4], fts[5], fts[6], fts[7]], y_test),
                  callbacks=[earlyStopping, reduce_lr_loss,history])
      print('Savig the model')
      base_path = "drive/My Drive/" if RunningInCOLAB else ""
      model.save(base_path + "fights_"+input_format+"_"+str(it)+".keras")

      v_input_shape = (1,) + input_shape
      input_signature = np.repeat(tf.TensorSpec(v_input_shape, tf.float32, name='inp1'), nbr_frame)
      model.output_names=['output']
      #onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=18)
      #onnx.save(onnx_model, base_path + "fights_tf_"+"_"+input_format+str(it)+".onnx")
      #open("drive/My Drive/fights"+str(it)+".tflite", "wb").write(tflite_model)

    #0.8995 4

Started at = 00:04:46
#### IT 0
Loading trainm from 0 to 150
Readed 150
Readed 300
(8, 300, 300, 420, 3)
(300, 300, 420, 3)
Epoch 1/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - accuracy: 0.4686 - loss: 0.7691 - val_accuracy: 0.5167 - val_loss: 0.6818 - learning_rate: 0.0010
Epoch 2/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - accuracy: 0.5565 - loss: 0.6713 - val_accuracy: 0.7500 - val_loss: 0.6219 - learning_rate: 0.0010
Epoch 3/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - accuracy: 0.6391 - loss: 0.6306 - val_accuracy: 0.7167 - val_loss: 0.5744 - learning_rate: 0.0010
Epoch 4/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - accuracy: 0.6844 - loss: 0.5896 - val_accuracy: 0.8167 - val_loss: 0.4884 - learning_rate: 0.0010
Epoch 5/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 163ms/step - accuracy: 0.7161 - loss: 0.5402 - val_accuracy: 0.8167 - val_loss: 0.4133 - learning_rate: 0.0010
Epoch 6/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 163ms/step - accuracy: 0.7851 - loss: 0.4644 - val_accuracy: 0.8500 - val_loss: 0.

KeyboardInterrupt: 

In [ ]:
model.save("fights_"+input_format+"_"+str(it)+".keras")

In [ ]:
print(np.array([_fights_train[0][0]]).shape)
model.predict( [ np.array([_fights_train[0][0]]),
                 np.array([_fights_train[1][0]]),
                 np.array([_fights_train[2][0]]),
                 np.array([_fights_train[3][0]]),
                 np.array([_fights_train[4][0]]),
                 np.array([_fights_train[5][0]]),
                 np.array([_fights_train[6][0]]),
                 np.array([_fights_train[7][0]]),])

In [ ]:
print('Savig the model')
model.save("drive/My Drive/fights"+str(it)+".keras")
v_input_shape = (1,) + input_shape
input_signature = []
for i in range(nbr_frame):
  input_signature.append(tf.TensorSpec(v_input_shape, tf.float32, name='inp'+str(i+1)))
model.output_names=['output']
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=17)
onnx.save(onnx_model, "drive/My Drive/fights_tf"+str(it)+".onnx")


In [ ]:
model.save("fights"+str(it)+".keras")

In [ ]:
# This is important, let's fix the input size.
v_input_shape = (0,) + full_input_shape


In [ ]:
#model = tf.keras.models.load_model('fights.keras')

In [ ]:
!ls -l -a /tmp

In [ ]:
  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  converter.target_spec.supported_ops = [
    #tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    #tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
  ]
  converter.experimental_new_converter = True
  converter.experimental_enable_resource_variables = True
  tflite_model = converter.convert()
  #open("fights"+str(it)+".tflite", "wb").write(tflite_model)
  print('ff')
  open("fights.tflite", "wb").write(tflite_model)
  #open("drive/My Drive/fights.tflite", "wb").write(tflite_model)

In [ ]:
#model = tf.keras.models.load_model('fights.keras')
interpreter = tf.lite.Interpreter(model_path='fights.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
import sys
import rknn.api

In [ ]:
TEST_CASES = 10
rknn = RKNN(verbose=True)
ret = rknn.init_runtime()
rknn.load_rknn('fights.rknn')
for i in range(TEST_CASES):
  inputs = [np.expand_dims(_fights_train[0][i], 0),
                            np.expand_dims(_fights_train[1][i], 0),
                            np.expand_dims(_fights_train[2][i], 0),
                            np.expand_dims(_fights_train[3][i], 0),
                            np.expand_dims(_fights_train[4][i], 0),
                            np.expand_dims(_fights_train[5][i], 0),
                            np.expand_dims(_fights_train[6][i], 0),
                            np.expand_dims(_fights_train[7][i], 0),]
  outputs = rknn.inference(inputs=inputs, data_format=['nhwc'])
  expected = model.predict([np.expand_dims(_fights_train[0][i], 0),
                            np.expand_dims(_fights_train[1][i], 0),
                            np.expand_dims(_fights_train[2][i], 0),
                            np.expand_dims(_fights_train[3][i], 0),
                            np.expand_dims(_fights_train[4][i], 0),
                            np.expand_dims(_fights_train[5][i], 0),
                            np.expand_dims(_fights_train[6][i], 0),
                            np.expand_dims(_fights_train[7][i], 0),])
  for j in range(8):
    interpreter.set_tensor(input_details[j]['index'], np.expand_dims(_fights_train[j][i], 0))
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])
  print(expected)
  print(outputs)
  print(result)
  #np.testing.assert_almost_equal(expected, result, decimal=5)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")
  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()

In [ ]:
!pip install git+https://github.com/onnx/tensorflow-onnx

In [ ]:
model = tf.keras.models.load_model('fights0.keras')

In [ ]:
!pip install git+https://github.com/onnx/tensorflow-onnx

In [ ]:
import tf2onnx
import onnx

In [ ]:
v_input_shape = (1,) + input_shape
print(v_input_shape)
input_signature = [tf.TensorSpec(v_input_shape, tf.float32, name='inp1'),
                   tf.TensorSpec(v_input_shape, tf.float32, name='inp2'),
                   tf.TensorSpec(v_input_shape, tf.float32, name='inp3'),
                   tf.TensorSpec(v_input_shape, tf.float32, name='inp4'),
                   tf.TensorSpec(v_input_shape, tf.float32, name='inp5'),
                   tf.TensorSpec(v_input_shape, tf.float32, name='inp6'),
                   tf.TensorSpec(v_input_shape, tf.float32, name='inp7'),
                   tf.TensorSpec(v_input_shape, tf.float32, name='inp8'),
                   ]
# Use from_function for tf functions
model.output_names=['output']
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=17)
onnx.save(onnx_model, "fights.onnx")

In [ ]:
onnx_model, _ = tf2onnx.convert.from_tflite('fights0.tflite', opset=12)
onnx.save(onnx_model, "fights-tf.onnx")

In [ ]:
rknn = RKNN(verbose=False)
means = [2,2,2]
#means = np.repeat(means[np.newaxis,...], 180, axis=0)
#means = np.repeat(means[np.newaxis,...], 150, axis=0)

rknn.config( target_platform='rv1106', mean_values=[means], std_values=[means], data_format='')
ret = rknn.load_onnx(model='fights.onnx', inputs=['x'], input_size_list=[[10, 150, 180, 3]])
ret = rknn.build(do_quantization=True, dataset='./dataset/dataset.txt')
ret = rknn.export_rknn('out.rknn')

In [ ]:
acc = history.acc
val_acc = history.val_acc
loss = history.loss
val_loss = history.val_loss
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
score = model.evaluate(fights_test, y_test, batch_size=3)
score

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

In [ ]:
Y_pred = model.predict(fights_test , batch_size=1)

In [ ]:
yprd = Y_pred > 0.5
yprd

In [ ]:
ypredicted = []
for zero,one in yprd:
    if zero == True:
        ypredicted.append(0)
    else:
        ypredicted.append(1)

In [ ]:
ypredicted

In [ ]:
y_test

In [ ]:
y = []

for zero,one in y_test:
    if zero == True:
        y.append(0)
    else:
        y.append(1)

In [ ]:
confusion = confusion_matrix(y,ypredicted)
confusion.shape

In [ ]:
print_confusion_matrix(confusion, [0,1], figsize = (30,15), fontsize=16)

In [ ]:
print('Classification Report')
print(classification_report(y, ypredicted, target_names=['no-violance','violance']))